index


gender: 성별


car: 차량 소유 여부


reality: 부동산 소유 여부


child_num: 자녀 수


income_total: 연간 소득


income_type: 소득 분류


['Commercial associate', 'Working', 'State servant', 'Pensioner', 'Student']



edu_type: 교육 수준


['Higher education' ,'Secondary / secondary special', 'Incomplete higher', 'Lower secondary', 'Academic degree']



family_type: 결혼 여부


['Married', 'Civil marriage', 'Separated', 'Single / not married', 'Widow']



house_type: 생활 방식


['Municipal apartment', 'House / apartment', 'With parents',

'Co-op apartment', 'Rented apartment', 'Office apartment']



DAYS_BIRTH: 출생일


데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 하루 전에 태어났음을 의미



DAYS_EMPLOYED: 업무 시작일(365243)


데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 하루 전부터 일을 시작함을 의미

양수 값은 고용되지 않은 상태를 의미함



FLAG_MOBIL: 핸드폰 소유 여부


work_phone: 업무용 전화 소유 여부


phone: 전화 소유 여부


email: 이메일 소유 여부


occyp_type: 직업 유형


family_size: 가족 규모


begin_month: 신용카드 발급 월
데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 한 달 전에 신용카드를 발급함을 의미

credit: 사용자의 신용카드 대금 연체를 기준으로 한 신용도


=> 낮을 수록 높은 신용의 신용카드 사용자를 의미함

In [1]:
import warnings
warnings.filterwarnings('ignore')
import glob
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
import random

In [2]:
train = pd.read_csv('train.csv')
train = train.drop(['index'], axis=1)
train.fillna('NAN', inplace=True) 


test = pd.read_csv('test.csv')
test = test.drop(['index'], axis=1)
test.fillna('NAN', inplace=True)

submit = pd.read_csv('submission.csv')

In [3]:
train.groupby(['occyp_type']).count() ##각 직업별 임금 평균 구하기
train_count = train.groupby(['occyp_type'])['gender'].count()
train.groupby(['occyp_type']).count()
train_total = train.groupby(['occyp_type'])['income_total'].sum()
train_avg = train_total/train_count
train_avg

occyp_type
Accountants              204651.942905
Cleaning staff           148209.677419
Cooking staff            146275.601751
Core staff               191473.629252
Drivers                  210853.714286
HR staff                 186387.096774
High skill tech staff    196602.403846
IT staff                 196770.731707
Laborers                 180307.189827
Low-skill Laborers       135708.661417
Managers                 278735.249654
Medicine staff           167212.364583
NAN                      169174.356444
Private service staff    198018.518519
Realty agents            246357.142857
Sales staff              174955.579756
Secretaries              163716.494845
Security staff           180252.594340
Waiters/barmen staff     159822.580645
dtype: float64

In [34]:
train_nan = train.loc[train.occyp_type =='NAN']
train_nan_un =train_nan.loc[train.DAYS_EMPLOYED>365000]
train_nan_un.groupby(['income_type']).count() #8171의 데이터 중 4438명의 경우 연금수령자이며 일하고 있지 않음

,gender,car,reality,child_num,income_total,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
income_type,,,,,,,,,,,,,,,,,,
Pensioner,4438,4438,4438,4438,4438,4438,4438,4438,4438,4438,4438,4438,4438,4438,4438,4438,4438,4438


In [46]:
train_nan = train.loc[train.occyp_type =='NAN']
train_nan_em =train_nan.loc[train.DAYS_EMPLOYED<0]
train_nan_em.groupby(['income_type']).count() 
#3733명의 데이터 중 1026: Commercial associate, 2:Pensioner, 392: State servant, 1: student 2312:working 

,gender,car,reality,child_num,income_total,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
income_type,,,,,,,,,,,,,,,,,,
Commercial associate,1026,1026,1026,1026,1026,1026,1026,1026,1026,1026,1026,1026,1026,1026,1026,1026,1026,1026
Pensioner,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
State servant,392,392,392,392,392,392,392,392,392,392,392,392,392,392,392,392,392,392
Student,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Working,2312,2312,2312,2312,2312,2312,2312,2312,2312,2312,2312,2312,2312,2312,2312,2312,2312,2312


In [47]:
train_nnan = train.loc[train.occyp_type !='NAN'] 
train_nnan_em =train_nnan.loc[train.DAYS_EMPLOYED<0]
train_nnan_em.groupby(['income_type']).count() 
#18286명의 데이터 중 5176: Commercial associate, 9:Pensioner, 1762: State servant, 6: student 11333:working 

,gender,car,reality,child_num,income_total,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
income_type,,,,,,,,,,,,,,,,,,
Commercial associate,5176,5176,5176,5176,5176,5176,5176,5176,5176,5176,5176,5176,5176,5176,5176,5176,5176,5176
Pensioner,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9
State servant,1762,1762,1762,1762,1762,1762,1762,1762,1762,1762,1762,1762,1762,1762,1762,1762,1762,1762
Student,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
Working,11333,11333,11333,11333,11333,11333,11333,11333,11333,11333,11333,11333,11333,11333,11333,11333,11333,11333


In [4]:
train_nnan_p = train.loc[train.income_type =='Student'] 
train_nnan_pem =train_nnan_p.loc[train_nnan_p.DAYS_EMPLOYED<0] 
train_nnan_pem #동일인물인데 credit이 다른 것을 확인할 수 있음 ->이에 대한 처리?

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
2990,F,N,Y,0,171000.0,Student,Higher education,Married,House / apartment,-18756,-3904,1,0,0,1,Core staff,2.0,-60.0,2.0
3602,F,N,Y,0,171000.0,Student,Higher education,Married,House / apartment,-18756,-3904,1,0,0,1,Core staff,2.0,-39.0,1.0
6223,M,N,N,0,135000.0,Student,Higher education,Single / not married,With parents,-9115,-581,1,0,0,1,NAN,1.0,-14.0,2.0
8936,F,N,Y,0,171000.0,Student,Higher education,Married,House / apartment,-18756,-3904,1,0,0,1,Core staff,2.0,-39.0,1.0
11646,F,N,Y,1,112500.0,Student,Secondary / secondary special,Civil marriage,House / apartment,-12127,-795,1,0,0,0,Laborers,3.0,-12.0,2.0
21681,F,N,Y,0,171000.0,Student,Higher education,Married,House / apartment,-18756,-3904,1,0,0,1,Core staff,2.0,-35.0,1.0
23119,F,N,Y,1,112500.0,Student,Secondary / secondary special,Civil marriage,House / apartment,-12127,-795,1,0,0,0,Laborers,3.0,-14.0,2.0


In [25]:
traind=train.groupby(['DAYS_BIRTH', 'DAYS_EMPLOYED','child_num']).count() #26457
traind

,,,gender,car,reality,income_total,income_type,edu_type,family_type,house_type,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
DAYS_BIRTH,DAYS_EMPLOYED,child_num,,,,,,,,,,,,,,,,
-25152,365243,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
-25140,365243,0,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
-25099,365243,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
-25088,365243,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
-24970,365243,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
-8041,-1419,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
-7959,-1045,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
-7757,-900,0,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3


In [27]:
traind.loc[traind.gender > 1] #5479명의 경우 카드 재발급 or 카드 여러개 발급으로 예상

,,,gender,car,reality,income_total,income_type,edu_type,family_type,house_type,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
DAYS_BIRTH,DAYS_EMPLOYED,child_num,,,,,,,,,,,,,,,,
-25140,365243,0,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
-24970,365243,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
-24946,365243,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
-24914,365243,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
-24843,365243,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
-8076,-596,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
-8056,-433,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
-8041,-1419,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2


In [18]:
test.groupby(['DAYS_BIRTH', 'DAYS_EMPLOYED']).count() #10000

,,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month
DAYS_BIRTH,DAYS_EMPLOYED,,,,,,,,,,,,,,,,
-25152,365243,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
-25010,365243,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
-24963,365243,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
-24946,365243,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
-24932,365243,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
-8041,-1419,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
-7980,-597,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
-7959,-1045,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [19]:
train.loc[train.family_size-train.child_num<0] #잘못된 데이터 child_num이 더 큼

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
14900,M,Y,N,2,225000.0,Working,Secondary / secondary special,Married,House / apartment,-14776,-2212,1,0,0,0,Drivers,1.0,-30.0,2.0


In [15]:
object_col = []
for col in train.columns:
    if train[col].dtype == 'object':
        object_col.append(col)

In [4]:
enc = OneHotEncoder()
enc.fit(train.loc[:,object_col])


train_onehot_df = pd.DataFrame(enc.transform(train.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
train.drop(object_col, axis=1, inplace=True)
train = pd.concat([train, train_onehot_df], axis=1)

In [ ]:
test_onehot_df = pd.DataFrame(enc.transform(test.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
test.drop(object_col, axis=1, inplace=True)
test = pd.concat([test, test_onehot_df], axis=1)

In [5]:
train['DAYS_BIRTH']= train['DAYS_BIRTH']*(-1/365)
test['DAYS_BIRTH']= test['DAYS_BIRTH']*(-1/365)

In [6]:
train['DAYS_EMPLOYED'].loc[train['DAYS_EMPLOYED']>0]=0
train['DAYS_EMPLOYED'].loc[train['DAYS_EMPLOYED']<0]= train['DAYS_EMPLOYED'].loc[train['DAYS_EMPLOYED']<0]*(-1/365)
test['DAYS_EMPLOYED'].loc[test['DAYS_EMPLOYED']>0]=0
test['DAYS_EMPLOYED'].loc[test['DAYS_EMPLOYED']<0]= test['DAYS_EMPLOYED'].loc[test['DAYS_EMPLOYED']<0]*(-1/365)

In [7]:
train['begin_month']= train['begin_month']*(-1/12)
test['begin_month']= test['begin_month']*(-1/12)

In [16]:
train

,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month,...,occyp_type_Low-skill Laborers,occyp_type_Managers,occyp_type_Medicine staff,occyp_type_NAN,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff
0,0,202500.0,38.079452,12.901370,1,0,0,0,2.0,0.500000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,247500.0,31.178082,4.219178,1,0,0,1,3.0,0.416667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,450000.0,52.293151,12.147945,1,0,1,0,2.0,1.833333,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,202500.0,41.336986,5.731507,1,0,1,0,2.0,3.083333,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0,157500.0,41.197260,5.767123,1,0,0,0,2.0,2.166667,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,2,225000.0,33.093151,5.435616,1,0,0,0,4.0,0.166667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26453,1,180000.0,41.893151,6.780822,1,0,0,0,2.0,3.916667,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
26454,0,292500.0,27.621918,5.520548,1,0,0,0,2.0,2.083333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26455,0,171000.0,27.794521,0.293151,1,0,0,0,1.0,4.916667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in skf.split(train, train['credit']):
    folds.append((train_idx, valid_idx))

In [10]:
random.seed(42)
lgb_models={}
for fold in range(5):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                         train['credit'][train_idx].values, train['credit'][valid_idx].values 
    lgb = LGBMClassifier(n_estimators=1000)
    lgb.fit(X_train, y_train, 
            eval_set=[(X_train, y_train), (X_valid, y_valid)], 
            early_stopping_rounds=30,
           verbose=100)
    lgb_models[fold]=lgb
    print(f'================================================================================\n\n')

====================================1============================================
Training until validation scores don't improve for 30 rounds
[100]	training's multi_logloss: 0.65115	valid_1's multi_logloss: 0.750211
[200]	training's multi_logloss: 0.565182	valid_1's multi_logloss: 0.731945
[300]	training's multi_logloss: 0.504345	valid_1's multi_logloss: 0.729441
Early stopping, best iteration is:
[278]	training's multi_logloss: 0.516279	valid_1's multi_logloss: 0.728845


====================================2============================================
Training until validation scores don't improve for 30 rounds
[100]	training's multi_logloss: 0.646185	valid_1's multi_logloss: 0.762279
[200]	training's multi_logloss: 0.55956	valid_1's multi_logloss: 0.750029
Early stopping, best iteration is:
[227]	training's multi_logloss: 0.54101	valid_1's multi_logloss: 0.74773


====================================3============================================
Training until validation scores don'

In [11]:
submit.iloc[:,1:]=0
for fold in range(5):
    submit.iloc[:,1:] += lgb_models[fold].predict_proba(test)/5

In [12]:
submit.to_csv('20210511_test_submit_ensemble.csv', index=False) # 0.7272812144